# Create a Simple Crypto Coin Arbitrage Bot Between Two Exchanges: Bitfinex and Binance

### Purpose of this tutorial
To create an augmented trading tool that will find, and alert you to, arbitrage opportunities across all assets on 2 different exchanges.
    
### What you will be creating:
An augmented trading tool that identifies realtime arbitrage opportunies across 2 exchanges that:
  - monitors both exchanges and all of their shared assets in real time
  - takes a `percent_difference_threshold` parameter that will filter our arbitrage opportunities to those with values that are at least X% different
  - displays a dataframe of coins with arbitrage opportunies that contains:
      - name of coin
      - price on bitfinex
      - price on binance
      - the percent difference between the two assets
      - and the 'recipe' for exploiting the opportunity:
     - e.g recipe: `BUY ETHUSD on BITFINEX and SELL ETHUSD on BINANCE`
  - updates this dataframe, checks for arbitrage opportunies, and prints the dataframe/recipe list every 7 seconds

    
### Prerequistes:
- Python3
- `livedataframe` package: `pip3 install livedataframe`
- public and secret keys from https://app.livedataframe.com/users/new (sign up is free)
- a can-do attitude

    
    
    


## Introduction To Arbitrage

#### What is Arbitrage?
Arbitrage is where you look at the price of an identical asset (in this case crypto coins) on two different exchanges and exploit a price difference (an arbitrage opportunity).

#### How will we be using it?
We will be looking at the price of identical crypto coins on Bitfinex and Binance and determining if an asset is trading at a higher price on a given exchange. We will _not_ be considering transfer/withdrawl fees, or trade/order fees in this tutorial. We will be making a tool that will automatically alert us when one of these opporunties arises.




## Lets Get Started


### Import Required Packages

- If you have not installed livedataframe please do so now: `pip3 install livedataframe`

In [ ]:
from livedataframe import LiveExchange, ExchangeInfo # The core packages used by this tutorial
import pandas as pd # import pandas for creating our new tool
from IPython.display import clear_output # for displaying the output of our monitoring tool
import time # for sleeping our tools loop

### Add your LiveDataFrame Api Keys
In the box below, please replace `<YOUR_PUBLIC_KEY_HERE>` and `<YOUR_SECRET_KEY_HERE>` with your actual public and secret LiveDataFrame keys (as strings). 

These Api keys should have been emailed to you when you  when you [signed up](https://app.livedataframe.com/users/new) for LiveDataFrane

In [ ]:
PUBLIC_KEY = "<YOUR_PUBLIC_KEY_HERE>"
SECRET_KEY = "<YOUR_SECRET_KEY_HERE>"

## Prepare the symbols we want to watch
Bitfinex has asset/base pairs that Binance does not have, and vice versa. The first thing we have to do is curate a list of asset/base pairs shared between each exchange. 



### Find the Bases (Markets) shared between Bitfinex and Binance

Let's start by getting a list of the base currencies used on each exchange.

In order to do this we will create two functions: one for getting a base from a pair and another for returning a list of bases from a list of pairs.

To get these lists of pairs we will utilize the helper class `ExchangeInfo` and its method `list_symbols_for`. This method will return a list of actively traded _pairs_ for that exchange. Once we have this list of pairs we will have to pull out the portion referencing the base currency.

Conveniently, both binance and bitfinex use the format `QUOTEBASE` (e.g the symbol `ETHUSD` corresponds to a quote asset of `ETH` (ethereum) and a base asset of `USD` (U.S Dollar). So, in order to get the base currency we just have to look at the tail end of the symbol returned. 

**GOTCHA:**
Usually the base currency is the last 3 characters of our symbol (as in the example above); however, there is one base that is 4 letters long: `USDT`. We need to manually check for `USDT` before we take the last 3 characters as the base - in the case of `USDT` we need to take the last 4 characters.

#### Create Functions to Get Base Currencies From a List of Pairs


In [ ]:
def get_base_from_pair(pair):
        # USDT is the only 4 letter base currency so we should manually check for it.
        # if found lets manually put it into the base list instead of taking the last 3 characters
        if 'USDT' in pair:
            return 'USDT'  
        else:
            return pair[-3:] # use the last 3 characters as the base currency identifier
            
def get_base_currencies_from_pairs(pairs):
    bases = [] # create empty list to store bases
    for pair in pairs: # iterate through all pairs returned
        base = get_base_from_pair(pair) # get base
        bases.append(base)

    return list(set(bases))# turn list into a set to get unique values and then back into a list

#### Get List of Base Currencies on Bitfinex

In [ ]:
bitfinex_pairs = ExchangeInfo.list_symbols_for('bitfinex')
bitfinex_bases = get_base_currencies_from_pairs(bitfinex_pairs)
bitfinex_bases

#### Get a List of Base Currencies on Binance

In [ ]:
binance_pairs = ExchangeInfo.list_symbols_for('binance')
binance_bases = get_base_currencies_from_pairs(binance_pairs)
binance_bases

#### Get a List of Shared Base Currencies
Now that we have a list of bases for each exchange, let's find out which bases are shared between the two exchanges. These shared bases will be the markets our arbitrage bot will operate in.

In [ ]:
shared_bases = list(set(binance_bases).intersection(bitfinex_bases))
shared_bases

At the time of writing there were only two shared bases: `ETH` and `BTC` so those are the only two markets that our arbitrage bot/script will operate in

### Curate the List of Shared Pairs from the Shared Bases (markets)
Now that we have a list of the shared bases (markets) on the two exchanges lets get a list of all the pairs shared between binance and bitfinex with these shared bases.

To start, let's create another function to select pairs with a specific base. This function will take a list of pairs and a specified list of bases to filter by. If a pair has one of the specified bases then it will be included in the output. If not, it will be ignored (filtered).


#### Create Function To Select Only Pairs that have the Specified Base Currencies

In [ ]:
def select_pairs_with_specified_base_currencies(specified_base_currencies, pairs):
    filtered_pairs = [] # create empty list to put pairs with specified base_currency into
    
    for specified_base_currency in specified_base_currencies: # itereate through list of specified base currencies
        for pair in pairs: # iterate through pairs
            base = get_base_from_pair(pair)

            if base == specified_base_currency: # check to see if pair has the specified base we are looking for
                filtered_pairs.append(pair) # add pair to filtered pairs list because it has correct base
            
    return filtered_pairs

#### Get Bitfinex Pairs for Shared Bases (Markets)

In [ ]:
bitfinex_pairs_with_shared_bases = select_pairs_with_specified_base_currencies(shared_bases, bitfinex_pairs)
bitfinex_pairs_with_shared_bases

#### Get Binance Pairs for Shared Bases (Markets)

In [ ]:
binance_pairs_with_shared_bases = select_pairs_with_specified_base_currencies(shared_bases, binance_pairs)
binance_pairs_with_shared_bases

#### Find Shared Pairs (finally)

For each exchange, we have now identified the list of pairs (symbols) that are trading in the markets shared by both exchanges. The last step is to to turn these two lists into one list by finding the shared pairs in these shared markets.

To do this we simply find the intersection of the two lists with some fancy pythoning.

In [ ]:
pairs_to_arbitrage = list(set(binance_pairs_with_shared_bases).intersection(bitfinex_pairs_with_shared_bases))
pairs_to_arbitrage

## Create the Arbitrage Tool/Bot

We now have a list of pairs we can hunt for arbitrage opportunities on: `pairs_to_arbitrage`. The next step is build the tool itself. We will start by instantiating two instances of `LiveExchange`: one for Bitfinex and one for Binance. 

`LiveExchange` provides us with a dictionary of dataframes that updates every 5 seconds with new data (i.e LiveDataFrames). The keys on this dictionary are the pairs and the values are the LiveDataFrames - there is one key/value and hence one symbol/LiveDataFrame combo for every pair. 

We will dive into `LiveExchange` a little later. For now, let's just create the instances.

### Initialize the LiveExchange Instances for Both Exchanges
We will initialize the LiveExchange instances with a 4 hour lookback period and our list of pairs to arbitrage. This means that for each pair we are watching (from the `pairs_to_arbitrage` list we calculated earlier) there will be a LiveDataFrame with 4 hours of historical data in it.


#### Initialize the LiveExchange Instance for Bitfinex



In [ ]:
live_bitfinex = LiveExchange(
    exchange='bitfinex', 
    symbols = pairs_to_arbitrage, 
    public_key = PUBLIC_KEY, 
    secret_key = SECRET_KEY,
    lookback_period = '4h'
)

#### Initialize the LiveExchange Instance for Binance



In [ ]:
live_binance = LiveExchange(
    exchange='binance', 
    symbols = pairs_to_arbitrage, 
    public_key = PUBLIC_KEY, 
    secret_key = SECRET_KEY,
    lookback_period = '4h',
)

## Start the LiveExchange Instances

After we have initialized the live exchange instances `live_bitfinex` and `live_binance`, we must start them by calling `.start()` on each instance. Calling `.start()` loads the historical data, subscribes to websockets for each symbol, and then appends each new tick into its own dataframe giving you a live dataframe with a 4 hour window of data. Let's start each LiveExchange instance now.

Please note that, depending on your internet speed, when you first call .start() it can take anywhere from 3 seconds to 1 minute *per exchange* to load the historical data. This wait period is **only** for the first load up and you only have to do it at the start and never again... so please be patient with this initial load :)


### Start live_bitfinex

In [ ]:
live_bitfinex.start()

### Start live_binance

In [ ]:
live_binance.start()

## Access Your first LiveDataFrames
As mentioned above, each `LiveExchange` instance provides you with a dictionary of LiveDataFrames for every symbol you requested - in our case the list of coins we are going arbitrage. 

To access one of these LiveDataFrames use the `.symbols` attribute while specifying the pair. 

For example:
- to request the `NEOBTC` LiveDataFrame for Bitfinex: `live_bitfinex.symbols["NEOBTC"]` 
- to request the `NEOBTC` LiveDataFrame for Binance: `live_binance.symbols["NEOBTC"]` 


### Access `NEOBTC` LiveDataFrame for Binance:
This dataframe continues 4 hours of historical data for `NEOBTC` AND it automagically updates every 5 - 10 seconds with a new row (the most recent data). How cool is that?

Let's take a look at the entire dataframe for `NEOBTC` on Binance

In [ ]:
live_binance.symbols['NEOBTC']

#### Congratulations! You have just opened up your first LiveDataFrame!

This dataframe contains 4 hours of historical data for `NEOBTC` on Binance AND it automagically updates every 5 - 10 seconds with a new row (the most recent data).

PLUS, you can access LiveDataFrames just like it for every other symbol on the Binance exchange by using the same pattern... How cool is that?

### Access `NEOBTC` LiveDataFrame for Bitfinex:
Let's do the same thing for Bitfinex. This time though, we will call `.tail()` on our LiveDataFrame to view only the last 5 rows/updates. 


In [ ]:
live_bitfinex.symbols['NEOBTC'].tail()

#### Seeing is Believing
After you have done that, wait 5 - 10 seconds and then run the above cell again. Notice how there is new data being appended in the background - this is the **Live** portion of __Live__DataFrame


## Quick Recap
Before we get on with creating this tool, let's take a second to discuss what we have done so far.

1. We curated a list pairs/coins that are being traded on both the Binance and Bitfinex Exchange
2. We started a `LiveExchange` instance for each exchange that:
    - contains a dictionary of dataframes for every symbol in our curated list
    - has 4 hours of historical data for each coin it
    - updates every 5-10 seconds with new data
3. We accessed our first LiveDataFrame and watched it magically update with new data

## Create the Arbitrage Tool

Now that we have our data readily accesible, let's go ahead and create this tool. Remember, it needs to: 
- take two live_exchange instances and a `percent_difference_threshold` as parameters 
- continuously check for new opportunites when new data comes in (every 5-10 seconds)
- display the opportunites in a dataframe telling us how to exploit them


### Create function to calculate the percent difference between two values
Since we don't want our tool to be alerting us to arbitrage opportunies that aren't worth it (i.e  the % difference between the two asset prices is too low), we need to create a function that calculates the % difference between two values.

This function will take two numbers as parameters and return a float representing the % difference between them.

We will be using the percent difference formula [here](https://www.calculatorsoup.com/calculators/algebra/percent-difference-calculator.php)


In [ ]:
def calculate_percent_difference(value_1, value_2):
    difference = value_1 - value_2
    average = (value_1 + value_2) / 2.0
    
    percent_difference = (abs(difference) / average) * 100
    
    return percent_difference

### Create a function to look for arbitrage opportunies across each exchange
This function will be the 'core' of our simple arbitrage tool. It will take two `LiveExchange` instances, a `percent_difference_threshold` value, and a list of `pairs_to_arbitrage` as arguments, look for opportunities across all assets on each exchange, and return a simple output containing:
    - name of coin/pair with the opportunity
    - price on bitfinex
    - price on binance
    - the percent difference between the two assets
    - and the 'recipe' for exploiting the opportunity:


In [ ]:
def find_arbitrage_opportunities(exchange_1, exchange_2, pairs_to_arbitrage, percent_difference_threshold):
    output = [] # create empty list to store output in
    

    for pair in pairs_to_arbitrage: # loop through pairs we are arbitraging
        # get the latest price on each exchange for the asset
        exchange_1_price = exchange_1.symbols[pair]['last_price'][-1] 
        exchange_2_price = exchange_2.symbols[pair]['last_price'][-1] 
        
        # calculate percent difference
        percent_difference = calculate_percent_difference(exchange_1_price, exchange_2_price)

        
        # check to see if percent difference is above threshold. Only process/add opportunity if it is
        if percent_difference > percent_difference_threshold:
            pair_data = { "pair": pair } # add symbol/pair to our output
            
            
            # Create recipe
            if exchange_1_price > exchange_2_price:
                recipe = "BUY on %s and SELL on %s" %(exchange_1.exchange.upper(), exchange_2.exchange.upper())
            else:
                recipe = "BUY on %s and SELL on %s" %(exchange_2.exchange.upper(), exchange_1.exchange.upper())

            
            
            pair_data["recipe"] = recipe   # Add recipe to output

                
            # add prices for each exchange to our output
            pair_data[exchange_1.exchange + '_last_price'] = exchange_1_price
            pair_data[exchange_2.exchange + '_last_price'] = exchange_2_price


            # add percent difference to our output
            pair_data["percent_difference"] = percent_difference
            
            output.append(pair_data)


    return output
            
        
    

### Let's run our function and check for percent differences!
Let's test out our new function by running it with `percent_threshold_difference = 2`. This will return all assets that currently have _at least_ a 2% difference in their `last_price` across the two exchanges.

In [ ]:
opportunities = find_arbitrage_opportunities(live_bitfinex, live_binance, pairs_to_arbitrage, percent_difference_threshold = 2)
opportunities

### Let's clean up the output and add our recipe

As we can see in the output above, there are quite a few opportunities to exploit at a 2% `last_price` difference. While this output is a great start, let's go ahead and write a function to formout this output into an easy-to-read dataframe that:
    - ranks the opportunities in descending order; and
    - adds the exploitation 'recipe'

In [ ]:
def format_arbitrage_opportunities(opportunities):
    output_df = pd.DataFrame(opportunities) # turn array of opportunities into dataframe
    output_df.index = output_df['pair'] #make index our 'pair' name
    output_df = output_df.drop('pair', axis =1) # drop extraneous pair column
    
    # sort percent differences in ascending order
    output_df = output_df.sort_values('percent_difference', ascending = False) 
    
    return output_df 
        

### Let's take a look at our pretty output

In [ ]:
format_arbitrage_opportunities(opportunities)

## Putting it All Together: Automation
Looking pretty sweet, huh? Now, whenever we want to find some arbitrage opportunities between bitfinex and binance we just have to run our cool new function. However, we are algorithm traders and we automate things. Let's go ahead and put it all together into a function that will automate this entire process and display a new set of results every 7 seconds. 

We will call this function `watch_markets_for_arbitrage_opportunitities`. It will take our two `live_exchange` instances, our `percent_difference_threshold`, and our list of `pairs_to_arbitrage` as arguments. It will output our prettified results dataframe and update it every 7 seconds with a fresh batch of opportunities matching our criteria.

In [ ]:
def watch_markets_for_arbitrage_opportunities(exchange_1, exchange_2, pairs_to_arbitrage, percent_difference_threshold):
    while True:
        # find opportunities
        opportunities = find_arbitrage_opportunities(exchange_1, exchange_2, pairs_to_arbitrage, percent_difference_threshold)
        
        # format opportunities
        formatted_opportunities = format_arbitrage_opportunities(opportunities)
        
        # clear jupyter's output - wait until we give it something new to display
        clear_output(wait = True) 
        
        # display results dataframe 
        # (this allows us to update the same output in jupyter without having to print another one below it)
        display(formatted_opportunities) 
        

## Identify Arbitrage Opportunities in Realtime

Our last step is by far the best: we get to turn it on and watch our tool display arbitrage opportunities as they happen in realtime! Run the cell below and pay close attention: it will update every 7 seconds. Depending on the time of day and the current market state, you may see large fluctuations or very small movements between each update - but it IS updating... we promise. Just look closely and you can see the percent difference values changing. Happy hunting!


In [ ]:
watch_markets_for_arbitrage_opportunities(live_bitfinex, live_binance, pairs_to_arbitrage, percent_difference_threshold = 2)

## Next steps to try on your own
- Be 'smart' about the arbitrage opportunities you find. Look at the percent difference between the spread (bid/ask) on each exchange. In this example we looked at the last trade for an asset and not the price you could immediately sell it at (bid) or the price you could immediately buy it at (ask).
- Have the tool physically alert you through a text message, email, or sound when a `alert_threshold` is reached
- Try arbitraging across two different exchanges.
- Try abritraging across three different exchanges at the same time!
- Arbitrage across different base values (BUY ETHUSD on BITTREX and SELL ETHBTC on BINANCE)
- Paper trade an arbitrage strategy and see how your bot does. Be sure to take into account trading fees!
- Use jupyter/ipython widgets to create a GUI for using this tool (instead of typing in parameters)
- Be sure to check out https://github.com/LiveDataFrame/Tutorials for more tutorials utilizing live dataframe